# OpenAI Function Calling

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [2]:
import json

# Example hard coded function to return the same weather (API in prod)
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [3]:
# OpenAI Function definition
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"]
                },
            },
            "required": ["location"],
        },
    },
]

In [4]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston in Celcisu?",
    }
]

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)
response = model.invoke(messages, functions=functions)

In [6]:
json.loads(response.additional_kwargs["function_call"]["arguments"])

{'location': 'Boston, MA', 'unit': 'celsius'}

In [7]:
print("Content:", response.content)
print("Additional kwargs:", response.additional_kwargs["function_call"])
args = json.loads(response.additional_kwargs["function_call"]["arguments"])
json.loads(get_current_weather(**args))

Content: 
Additional kwargs: {'arguments': '{"location":"Boston, MA","unit":"celsius"}', 'name': 'get_current_weather'}


{'location': 'Boston, MA',
 'temperature': '72',
 'unit': 'celsius',
 'forecast': ['sunny', 'windy']}

- **Message that's not related to the function**

In [8]:
messages = [
    {
        "role": "user",
        "content": "Hi!",
    }
]
response = model.invoke(messages, functions=functions)

In [9]:
print("Content:", response.content)
print("Additional kwargs:", response.additional_kwargs)
# args = json.loads(response.additional_kwargs["function_call"]["arguments"])
# json.loads(get_current_weather(**args))

Content: Hello! How can I assist you today?
Additional kwargs: {'refusal': None}


- **Control function call**

In [10]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?",
    }
]
response = model.invoke(messages, functions=functions, function_call="none")

print("Content:", response.content)
print("Additional kwargs:", response.additional_kwargs)

Content: Which unit would you prefer for the temperature: Celsius or Fahrenheit?
Additional kwargs: {'refusal': None}


In [11]:
messages = [
    {
        "role": "user",
        "content": "Hi!",
    }
]
response = model.invoke(messages, functions=functions, function_call={"name": "get_current_weather"})

print("Content:", response.content)
print("Additional kwargs:", response.additional_kwargs)

Content: 
Additional kwargs: {'function_call': {'arguments': '{"location":"New York, NY"}', 'name': 'get_current_weather'}, 'refusal': None}


- **Respond after executing the function**

In [12]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?",
    }
]
response = model.invoke(messages, functions=functions, function_call={"name": "get_current_weather"})
messages.append(response)

args = json.loads(response.additional_kwargs["function_call"]["arguments"])
weather = get_current_weather(**args)

messages.append(
    {
        "role": "function",
        "name": "get_current_weather",
        "content": weather,
    }
)

response = model.invoke(messages)
response.content

'The current weather in Boston is 72°F, sunny, and windy.'